In [12]:
import functools
import jax
import jax.numpy as jnp
import jraph
import optax
import sys
import ml_collections
import e3nn_jax as e3nn
import ml_collections

sys.path.append('..')
sys.path.append('../analyses')
import analyses.analysis as analysis
from symphony.data import input_pipeline_tf, input_pipeline, matproj, fragments
from symphony import loss, train

In [9]:
structs = matproj.get_materials({"elements": ["O", "Si"], "num_elements": (2, 2)})
graphs = [input_pipeline.material_to_jraph_graph(struct.structure, [8, 14], (0.5, 1.5)) for struct in structs[:10]]

Retrieving SummaryDoc documents: 100%|██████████| 343/343 [00:00<00:00, 4910055.54it/s]


In [23]:
graph = graphs[0]
rng = jax.random.PRNGKey(0)
frag_rng, rng = jax.random.split(rng)
frags = list(fragments.generate_fragments(
    frag_rng,
    graph,
    2,
    max_targets_per_graph = 4,
    periodic = True
))

In [24]:
frags[0].globals.target_positions

array([[[-1.48560643, -0.6553905 ,  0.        ],
        [ 1.48561549, -0.6553905 ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]])